In [1]:
import pandas as pd
import numpy as np
from score_submission import our_scorer, scorer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn import datasets, linear_model
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestRegressor
import pickle

In [2]:
from functions.date_split import split_month_test
from score_submission import our_scorer, scorer
#import functions.CreatingNewDataframe as CreatingNewDataframe
import functions.GenerateFis as Features
import functions.GenerateMeansWithOffset as MeansOffset
from functions.pipeline_steps import *

## Data

In [3]:
df_test = pd.read_csv('train.csv.zip', parse_dates=['DATE'])
ids = np.unique(df_test.ATM_ID)

### Добавим даты для predict

In [4]:
pred_dates  = np.array(['2017-08-16', '2017-08-17', '2017-08-18', '2017-08-19',
               '2017-08-20', '2017-08-21', '2017-08-22', '2017-08-23',
               '2017-08-24', '2017-08-25', '2017-08-26', '2017-08-27',
               '2017-08-28', '2017-08-29', '2017-08-30', '2017-08-31',
               '2017-09-01', '2017-09-02', '2017-09-03', '2017-09-04',
               '2017-09-05', '2017-09-06', '2017-09-07', '2017-09-08',
               '2017-09-09', '2017-09-10', '2017-09-11', '2017-09-12',
               '2017-09-13', '2017-09-14', '2017-09-15', '2017-09-16',
               '2017-09-17'], dtype='datetime64')

data = []
for id in ids:
    for date in pred_dates:
        data.append((date, id, np.nan))
df_test_append = pd.DataFrame(data=data, columns=['DATE', 'ATM_ID', 'CLIENT_OUT'])
df_test = df_test.append(df_test_append)

### Создадим фичи

In [5]:
df1, df2, df3, df4, df5 = Features.GenerateFisSasha(df_test)

In [6]:
test_days = len(pred_dates)
#_, test_y = split_month_test(df_initial, train_days) # для теста данных нет
_, test1_X = split_month_test(df5, test_days)

In [7]:
#test1_X = test1_X.drop(['CLIENT_OUT'], axis=1)

In [8]:
#test1_X = test1_X.drop(['CLIENT_OUT'], axis=1)
#test1_X = test1_X.dropna(subset=test1_X.columns.drop('CLIENT_OUT'))

In [9]:
test1_X

,DATE,ATM_ID,CLIENT_OUT,weekend_dummy,holiday_dummy,day_of_week_0,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,...,same_day_8_week_ago,mean_8_week_ago,std_8_week_ago,same_day_8_week_ago_plusday,same_day_8_week_ago_minusday,same_day_9_week_ago,mean_9_week_ago,std_9_week_ago,same_day_9_week_ago_plusday,same_day_9_week_ago_minusday
0,2017-08-16,0,NaN,0.000000,0.0,0.000000,0.000000,399242.857143,0.000000,0.000000,...,543000.0,514814.285714,156600.249346,384100.0,675100.0,838500.0,520557.142857,263764.723805,692300.0,598800.0
1,2017-08-17,0,NaN,0.000000,0.0,0.000000,0.000000,0.000000,357628.571429,0.000000,...,384100.0,470785.714286,140923.897058,462000.0,543000.0,692300.0,525657.142857,267156.900573,597600.0,838500.0
2,2017-08-18,0,NaN,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,355528.571429,...,462000.0,451414.285714,129438.420733,230500.0,384100.0,597600.0,502971.428571,250522.079628,265700.0,692300.0
3,2017-08-19,0,NaN,353257.142857,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,230500.0,446385.714286,138240.218805,307400.0,462000.0,265700.0,480342.857143,265537.359940,402400.0,597600.0
4,2017-08-20,0,NaN,370142.857143,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,307400.0,432814.285714,147622.914081,740100.0,230500.0,402400.0,515014.285714,230184.356756,427600.0,265700.0
5,2017-08-21,0,NaN,0.000000,0.0,367485.714286,0.000000,0.000000,0.000000,0.000000,...,740100.0,477457.142857,187617.330908,453800.0,307400.0,427600.0,546128.571429,193914.858587,675100.0,402400.0
6,2017-08-22,0,NaN,0.000000,0.0,0.000000,358142.857143,0.000000,0.000000,0.000000,...,453800.0,445842.857143,166183.782373,369900.0,740100.0,675100.0,557028.571429,199434.832416,543000.0,427600.0
7,2017-08-23,0,NaN,0.000000,0.0,0.000000,0.000000,385014.285714,0.000000,0.000000,...,369900.0,421114.285714,162146.841151,702700.0,453800.0,543000.0,514814.285714,156600.249346,384100.0,675100.0
8,2017-08-24,0,NaN,0.000000,0.0,0.000000,0.000000,0.000000,372242.857143,0.000000,...,702700.0,466628.571429,191993.573801,873900.0,369900.0,384100.0,470785.714286,140923.897058,462000.0,543000.0
9,2017-08-25,0,NaN,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,386885.714286,...,873900.0,525471.428571,245893.071031,353600.0,702700.0,462000.0,451414.285714,129438.420733,230500.0,384100.0


## This one works. The simplest.

In [10]:
with open("model_to_run.pickle", "rb") as file:
       clf = pickle.load(file)

test_y_pred = clf.predict(test1_X)

WrapperForEstimator transform called
Transform from estimator Finished
300
FinalStepForSubmission Finished


## From Nastya

In [11]:
# dfs = Features.GenerateFis(df_test)
# atm0 = df5

# with open("lasso_n.pickle", "rb") as file:
#     clf = pickle.load(file)
    
# X_test = test1_X[atm0.columns.difference(['DATE', 'CLIENT_OUT'])]

# test_y_prediction = clf.predict(X_test)
# test_y_pred = test1_X[['DATE', 'ATM_ID']].copy()
# test_y_pred['CLIENT_OUT'] = test_y_prediction

In [12]:
# dfs = Features.GenerateFis(df_test)
# atm0 = df5

# with open("sgd.pickle", "rb") as file:
#     clf = pickle.load(file)
    
# X_test = test1_X[atm0.columns.difference(['DATE', 'CLIENT_OUT'])]

# test_y_prediction = clf.predict(X_test)
# test_y_pred = test1_X[['DATE', 'ATM_ID']].copy()
# test_y_pred['CLIENT_OUT'] = test_y_prediction

## From Nastya random forest

In [13]:
# dfs = Features.GenerateFis(df_test)
# atm0 = dfs[0]

# with open("rf_rcv.pickle", "rb") as file:
#     clf = pickle.load(file)
# X_test = test1_X[atm0.columns.difference(['DATE', 'CLIENT_OUT'])]

# test_y_prediction = clf.predict(X_test)
# test_y_pred = test1_X[['DATE', 'ATM_ID']].copy()
# test_y_pred['CLIENT_OUT'] = test_y_prediction

## Final steps

In [14]:
# convert negative values to zeros
test_y_pred['CLIENT_OUT'] = test_y_pred['CLIENT_OUT'].where(test_y_pred['CLIENT_OUT'] > 0, 0)

In [15]:
test_y_pred.to_csv('submission.csv', index=False)